In [4]:
import pandas as pd 

df=pd.read_csv("changed_data.csv")


In [5]:
df.head()

,type,posts
0,INFJ,moments sportscenter top ten plays pranks lif...
1,ENTP,finding lack posts alarming sex boring positio...
2,INTP,good one course say know blessing curse absolu...
3,INTJ,dear enjoyed conversation day esoteric gabbin...
4,ENTJ,fired another silly misconception approaching ...


In [6]:
def get_types(row):
    t=row['type']

    I = 0; N = 0
    T = 0; J = 0
    
    if t[0] == 'I': I = 1
    elif t[0] == 'E': I = 0
    else: print('I-E not found') 
        
    if t[1] == 'N': N = 1
    elif t[1] == 'S': N = 0
    else: print('N-S not found')
        
    if t[2] == 'T': T = 1
    elif t[2] == 'F': T = 0
    else: print('T-F not found')
        
    if t[3] == 'J': J = 1
    elif t[3] == 'P': J = 0
    else: print('J-P not found')
    return pd.Series( {'IE':I, 'NS':N , 'TF': T, 'JP': J }) 

df = df.join(df.apply (lambda row: get_types (row),axis=1))
df.head(5)

,type,posts,IE,NS,TF,JP
0,INFJ,moments sportscenter top ten plays pranks lif...,1,1,0,1
1,ENTP,finding lack posts alarming sex boring positio...,0,1,1,0
2,INTP,good one course say know blessing curse absolu...,1,1,1,0
3,INTJ,dear enjoyed conversation day esoteric gabbin...,1,1,1,1
4,ENTJ,fired another silly misconception approaching ...,0,1,1,1


In [9]:
# Splitting the MBTI personality into 4 letters and binarizing it
import numpy as np
b_Pers = {'I':0, 'E':1, 'N':0, 'S':1, 'F':0, 'T':1, 'J':0, 'P':1}
b_Pers_list = [{0:'I', 1:'E'}, {0:'N', 1:'S'}, {0:'F', 1:'T'}, {0:'J', 1:'P'}]

def translate_personality(personality):
    # transform mbti to binary vector
    return [b_Pers[l] for l in personality]

#To show result output for personality prediction
def translate_back(personality):
    # transform binary vector to mbti personality
    s = ""
    for i, l in enumerate(personality):
        s += b_Pers_list[i][l]
    return s

#list_personality_bin = np.array([translate_personality(p) for p in data.type])
#print("Binarize MBTI list: \n%s" % list_personality_bin)

In [17]:
import re
def pre_process_text(data, remove_stop_words=True, remove_mbti_profiles=True):
  list_personality = []
  list_posts = []
  len_data = len(data)
  i=0
  
  for row in data.iterrows():
      # check code working 
      # i+=1
      # if (i % 500 == 0 or i == 1 or i == len_data):
      #     print("%s of %s rows" % (i, len_data))

      #Remove and clean comments
      posts = row[1].posts

      #Remove url links 
      #temp = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', posts)

      #Remove Non-words - keep only words
      #temp = re.sub("[^a-zA-Z]", " ", temp)

      # Remove spaces > 1
      temp = re.sub(' +', ' ', posts).lower()
      temp = re.sub("[^a-zA-Z]", " ", temp)
      #Remove multiple letter repeating words
      #temp = re.sub(r'([a-z])\1{2,}[\s|\w]*', '', temp)

      #Remove stop words
      #if remove_stop_words:
      #    temp = " ".join([lemmatiser.lemmatize(w) for w in temp.split(' ') if w not in useless_words])
      #else:
      #    temp = " ".join([lemmatiser.lemmatize(w) for w in temp.split(' ')])
          
      #Remove MBTI personality words from posts
      #if remove_mbti_profiles:
      #    for t in unique_type_list:
      #        temp = temp.replace(t,"")

      # transform mbti to binary vector
      type_labelized = translate_personality(row[1].type) #or use lab_encoder.transform([row[1].type])[0]
      list_personality.append(type_labelized)
      # the cleaned data temp is passed here
      list_posts.append(temp)

  # returns the result
  list_posts = np.array(list_posts)
  list_personality = np.array(list_personality)
  return list_posts, list_personality

list_posts, list_personality  = pre_process_text(df, remove_stop_words=True, remove_mbti_profiles=True)

print("Example :")
#print("\nPost before preprocessing:\n\n", data.posts[0])
#print("\nPost after preprocessing:\n\n", list_posts[0])
print("\nMBTI before preprocessing:\n\n", df.type[0])
print("\nMBTI after preprocessing:\n\n", list_personality[0])



Example :

MBTI before preprocessing:

 INFJ

MBTI after preprocessing:

 [0 0 0 0]


In [18]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import sklearn.cluster as cluster
from sklearn.manifold import TSNE

cntizer = CountVectorizer(analyzer="word", 
                             max_features=1000,  
                             max_df=0.7,
                             min_df=0.1) 
# the feature should be made of word n-gram 
# Learn the vocabulary dictionary and return term-document matrix
print("Using CountVectorizer :")
X_cnt = cntizer.fit_transform(list_posts)

print(X_cnt)

Using CountVectorizer :
  (0, 712)	1
  (0, 211)	2
  (0, 707)	2
  (0, 426)	1
  (0, 510)	1
  (0, 370)	1
  (0, 254)	1
  (0, 534)	1
  (0, 470)	1
  (0, 148)	1
  (0, 579)	1
  (0, 639)	1
  (0, 302)	1
  (0, 460)	1
  (0, 571)	1
  (0, 200)	1
  (0, 446)	1
  (0, 724)	2
  (0, 239)	1
  (0, 298)	1
  (0, 705)	2
  (0, 764)	2
  (0, 666)	1
  (0, 264)	2
  (0, 605)	1
  :	:
  (8674, 149)	1
  (8674, 650)	1
  (8674, 659)	1
  (8674, 752)	1
  (8674, 486)	2
  (8674, 224)	2
  (8674, 479)	1
  (8674, 68)	2
  (8674, 369)	1
  (8674, 206)	1
  (8674, 363)	1
  (8674, 575)	3
  (8674, 61)	1
  (8674, 718)	1
  (8674, 671)	2
  (8674, 757)	2
  (8674, 95)	1
  (8674, 466)	1
  (8674, 216)	1
  (8674, 376)	1
  (8674, 644)	1
  (8674, 638)	1
  (8674, 596)	1
  (8674, 517)	1
  (8674, 293)	1


In [19]:
feature_names = list(enumerate(cntizer.get_feature_names_out()))
print("10 feature names can be seen below")
print(feature_names[0:10])

10 feature names can be seen below
[(0, 'ability'), (1, 'able'), (2, 'absolutely'), (3, 'accept'), (4, 'accurate'), (5, 'across'), (6, 'act'), (7, 'action'), (8, 'actual'), (9, 'actually')]


In [20]:
tfizer = TfidfTransformer()

# Learn the idf vector (fit) and transform a count matrix to a tf-idf representation
print("\nUsing Tf-idf :")

print("Now the dataset size is as below")
X_tfidf =  tfizer.fit_transform(X_cnt).toarray()
print(X_tfidf.shape)


Using Tf-idf :
Now the dataset size is as below
(8675, 805)


In [21]:
print(X_tfidf)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.04927502 0.         ... 0.         0.         0.        ]
 [0.15748878 0.05462647 0.13163268 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.06630823 0.         ... 0.         0.         0.        ]
 [0.         0.04007086 0.         ... 0.         0.0539018  0.        ]]


In [23]:
personality_type = [ "IE: Introversion (I) / Extroversion (E)", "NS: Intuition (N) / Sensing (S)", 
                   "FT: Feeling (F) / Thinking (T)", "JP: Judging (J) / Perceiving (P)"  ]

for l in range(len(personality_type)):
    print(personality_type[l])

IE: Introversion (I) / Extroversion (E)
NS: Intuition (N) / Sensing (S)
FT: Feeling (F) / Thinking (T)
JP: Judging (J) / Perceiving (P)


In [22]:
X = X_tfidf

In [26]:
import joblib
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score, balanced_accuracy_score
# Define a dictionary to store trained models
trained_models = {}

for l in range(len(personality_type)):
    Y = list_personality[:, l]

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=7)

    # Fit model on training data
    model = SVC(random_state=1)
    model.fit(X_train, y_train)

    # Save the trained model to a file
    model_filename = f"model_{l}.joblib"
    joblib.dump(model, model_filename)

    # Store the trained model in the dictionary
    trained_models[personality_type[l]] = model_filename

    # Make predictions for test data
    y_pred = model.predict(X_test)
    
    predictions = [round(value) for value in y_pred]
    
    # Evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    
    print("%s Accuracy: %.2f%%" % (personality_type[l], accuracy * 100.0))

# Print the filenames of the saved models
print("Trained models saved as:")
for personality, model_filename in trained_models.items():
    print(f"{personality}: {model_filename}")


IE: Introversion (I) / Extroversion (E) Accuracy: 78.52%
NS: Intuition (N) / Sensing (S) Accuracy: 86.03%
FT: Feeling (F) / Thinking (T) Accuracy: 76.35%
JP: Judging (J) / Perceiving (P) Accuracy: 66.82%
Trained models saved as:
IE: Introversion (I) / Extroversion (E): model_0.joblib
NS: Intuition (N) / Sensing (S): model_1.joblib
FT: Feeling (F) / Thinking (T): model_2.joblib
JP: Judging (J) / Perceiving (P): model_3.joblib


In [27]:
import joblib

# Save the CountVectorizer
joblib.dump(cntizer, 'count_vectorizer.pkl')

# Save the TfidfTransformer
joblib.dump(tfizer, 'tfidf_transformer.pkl')


['tfidf_transformer.pkl']

In [28]:
import pandas as pd
import nltk
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
nltk.download('stopwords')
nltk.download('wordnet')
import pandas as pd
#df=pd.read_csv('mbti_1.csv')

def preprocess_data(post):
    #data_length=[]
    lemmatizer=WordNetLemmatizer()
    #cleaned_text=[]
    stop_words = set(stopwords.words('english')) # Load stop words
    pers_types = ['INFP' ,'INFJ', 'INTP', 'INTJ', 'ENTP', 'ENFP', 'ISTP' ,'ISFP' ,'ENTJ', 'ISTJ','ENFJ', 'ISFJ' ,'ESTP', 'ESFP' ,'ESFJ' ,'ESTJ']
    pers_types = [p.lower() for p in pers_types]
    sentence= post
    print("Cleaning The Dataset")
    #for sentence in tqdm(df.posts):

    sentence=sentence.lower()

    sentence=re.sub('https?://[^\s<>"]+|www\.[^\s<>"]+',' ',sentence)

    sentence=re.sub('[^0-9a-z]',' ',sentence)

    sentence = " ".join([word for word in sentence.split() if word not in stop_words]) # Remove stop words
        #print(len(sentence))

    for p in pers_types:
        sentence = re.sub(p, '', sentence)
        #print(len(sentence))

    sentence = lemmatizer.lemmatize(sentence) # Lemmatize words
    #data_length.append(len(sentence.split())) #Split data, measure length of new filtered data

    cleaned_text= sentence
    return pd.DataFrame(data={'type': ['INFJ'], 'posts': [cleaned_text]})




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lazee\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lazee\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [29]:
import re
def pre_process_text(data, remove_stop_words=True, remove_mbti_profiles=True):
  list_personality = []
  list_posts = []
  len_data = len(data)
  i=0
  
  for row in data.iterrows():
      # check code working 
      # i+=1
      # if (i % 500 == 0 or i == 1 or i == len_data):
      #     print("%s of %s rows" % (i, len_data))

      #Remove and clean comments
      posts = row[1].posts

      #Remove url links 
      #temp = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', posts)

      #Remove Non-words - keep only words
      #temp = re.sub("[^a-zA-Z]", " ", temp)

      # Remove spaces > 1
      temp = re.sub(' +', ' ', posts).lower()
      temp = re.sub("[^a-zA-Z]", " ", temp)
      #Remove multiple letter repeating words
      #temp = re.sub(r'([a-z])\1{2,}[\s|\w]*', '', temp)

      #Remove stop words
      #if remove_stop_words:
      #    temp = " ".join([lemmatiser.lemmatize(w) for w in temp.split(' ') if w not in useless_words])
      #else:
      #    temp = " ".join([lemmatiser.lemmatize(w) for w in temp.split(' ')])
          
      #Remove MBTI personality words from posts
      #if remove_mbti_profiles:
      #    for t in unique_type_list:
      #        temp = temp.replace(t,"")

      # transform mbti to binary vector
      type_labelized = translate_personality(row[1].type) #or use lab_encoder.transform([row[1].type])[0]
      list_personality.append(type_labelized)
      # the cleaned data temp is passed here
      list_posts.append(temp)

  # returns the result
  list_posts = np.array(list_posts)
  list_personality = np.array(list_personality)
  return list_posts, list_personality

#list_posts, list_personality  = pre_process_text(df, remove_stop_words=True, remove_mbti_profiles=True)




In [37]:
my_posts = """ They act like they care They tell me to share But when I carve the stories on my arm The doctor just calls it self harm I’m not asking for attention There’s a reason I have apprehensions I just need you to see What has become of me||| I know I’m going crazy But they think my thoughts are just hazy When in that chaos, in that confusion I’m crying out for help, to escape my delusions||| Mental health is a state of mind How does one keep that up when assistance is denied All my failed attempts to fight the blaze You treat it like its a passing phase||| Well stop, its not, because mental illness is real Understand that we’re all not made of steel Because when you brush these issues under the carpet You make it seem like its our mistake we’re not guarded||| Don’t you realise that its a problem that needs to be addressed Starting at home, in our nest Why do you keep your mouths shut about such things Instead of caring for those with broken wings||| What use is this social stigma When mental illness is not even such an enigma Look around and you’ll see the numbers of the affected hiding under the covers ||| This is an issue that needs to be discussed Not looked down upon with disgust Mental illness needs to be accepted So that people can be protected ||| Let me give you some direction People need affection The darkness must be escaped Only then the lost can be saved||| Bring in a change Something not very strange The new year is here Its time to eradicate fear||| Recognise the wrists under the knives To stop mental illness from taking more lives Let’s break the convention Start ‘suicide prevention’.||| Hoping the festival of lights drives the darkness of mental illness away"""

# The type is just a dummy so that the data prep function can be reused
#mydata = pd.DataFrame(data={'type': ['INFJ'], 'posts': [my_posts]})
cleaned_text=preprocess_data(my_posts)

my_posts, dummy  = pre_process_text(cleaned_text, remove_stop_words=True, remove_mbti_profiles=True)
cntizer = joblib.load('count_vectorizer.pkl')

tfizer = joblib.load('tfidf_transformer.pkl')

my_X_cnt = cntizer.transform(my_posts)
my_X_tfidf =  tfizer.transform(my_X_cnt).toarray()


Cleaning The Dataset


In [40]:
# setup parameters for xgboost
from xgboost import XGBClassifier
from xgboost import plot_importance


param = {}
param['n_estimators'] = 200
param['max_depth'] = 2
param['nthread'] = 8
param['learning_rate'] = 0.2

#XGBoost model for MBTI dataset
result = []
# Individually training each mbti personlity type
for l in range(len(personality_type)):
    print("%s classifier trained" % (personality_type[l]))
    
    Y = list_personality[:,l]

    # split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=7)

    # fit model on training data
    model = XGBClassifier(**param)
    model.fit(X_train, y_train)
    
    # make predictions for my  data
    y_pred = model.predict(my_X_tfidf)
    result.append(y_pred[0])

IE: Introversion (I) / Extroversion (E) classifier trained
NS: Intuition (N) / Sensing (S) classifier trained
FT: Feeling (F) / Thinking (T) classifier trained
JP: Judging (J) / Perceiving (P) classifier trained


In [42]:
print(translate_back(result))

INTP


In [38]:
# Load the SVM model
svm_model_IE = joblib.load('model_0.joblib')
svm_model_NS = joblib.load('model_1.joblib')
svm_model_FT = joblib.load('model_2.joblib')
svm_model_JP = joblib.load('model_3.joblib')

# Make predictions using the loaded model
y_pred_IE = svm_model_IE.predict(my_X_tfidf)[0]
y_pred_NS = svm_model_NS.predict(my_X_tfidf)[0]
y_pred_FT = svm_model_FT.predict(my_X_tfidf)[0]
y_pred_JP = svm_model_JP.predict(my_X_tfidf)[0]

results=[y_pred_IE,y_pred_NS,y_pred_FT,y_pred_JP]


In [39]:
print(translate_back(results))

INTP
